In [49]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict, Annotated
from langgraph.types import Command

In [50]:
class State(TypedDict):
    transfer_reason: str

graph_builder = StateGraph(State)

In [51]:
def triage_node(state: State):
    return Command(
        goto="account_support",
        update={
            "transfer_reason": "The customer has an issue with their account."
        }
    )

def tech_support(state: State):
    return {}

def account_support(state: State):
    print("Handling account issue:", state["transfer_reason"])
    return {}

In [52]:
graph_builder.add_node("triage_node", triage_node)
graph_builder.add_node("tech_support", tech_support)
graph_builder.add_node("account_support", account_support)

graph_builder.add_edge(START, "triage_node")
graph_builder.add_edge("tech_support", END)
graph_builder.add_edge("account_support", END)

In [53]:
graph = graph_builder.compile()

graph.invoke({})

Handling account issue: The customer has an issue with their account.


{'transfer_reason': 'The customer has an issue with their account.'}